In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [3]:
df = pd.read_csv('NYC_Permitted_Event_Information_20250605.csv')

In [13]:
def parse_datetime(date_str, time_str):
    datetime_str = f"{date_str} {time_str}"
    return pd.to_datetime(datetime_str)

In [15]:
df['Start Datetime'] = df.apply(lambda row: parse_datetime(row['Start Date/Time'], row['Start Time']), axis=1)
df['End Datetime'] = df.apply(lambda row: parse_datetime(row['End Date/Time'], row['End Time']), axis=1)

In [17]:
df.dtypes

Event ID                                int64
Event Name                             object
Event Type                             object
Cultural activity suggested            object
Start Date/Time                        object
Start Time                             object
End Date/Time                          object
End Time                               object
Event Agency                           object
Event Borough                          object
Event Location                         object
Start Datetime                 datetime64[ns]
End Datetime                   datetime64[ns]
dtype: object

In [21]:
def get_first_location(location_str):
    return location_str.split(',')[0].strip()

In [23]:
df['Event Location'] = df['Event Location'].apply(get_first_location)

In [31]:
expanded_rows = []

In [33]:
for _, row in df.iterrows():
    current_time = row['Start Datetime']
    end_time = row['End Datetime']
    
    while current_time <= end_time:
        expanded_rows.append({
            'Event Type': row['Event Type'],
            'Event Location': row['Event Location'],
            'Cultural activity suggested': row['Cultural activity suggested'],
            'Date Time': current_time
        })
        current_time += timedelta(minutes=30)

In [35]:
expanded_df = pd.DataFrame(expanded_rows)

In [37]:
expanded_df = expanded_df.sort_values('Date Time')

In [39]:
expanded_df.to_csv('events_permit_modified.csv', index=False)